In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage
import requests

In [ ]:
# tool create
@tool
def multiply(a: int, b: int) -> int:
    """
    Given two numbers a and b this tool returns their product
    """
    return a * b

In [ ]:
print(multiply.invoke({'a':6, 'b': 5}))

In [ ]:
mulyiply.name
mulyiply.description
mulyiply.args

In [ ]:
llm = ChatOpenAI()
llm_with_tools = llm.bind_tools([multiply])

In [ ]:
llm_with_tools.invoke("Hi how are you?")

In [ ]:
query = HumanMessage('Can you multiply 3 with 10?')

In [ ]:
messages = [query]

In [ ]:
result = llm_with_tools.invoke(messages)

In [ ]:
messages.append(result)

In [ ]:
print(result)

In [ ]:
tool_result = multiply.invoke(result.tool_calls[0])

In [ ]:
messages.append(tool_result)

In [ ]:
# Currency conversion
# tool create
from langchain_core import InjectedToolArg
from typing import Annotated


@tool
def get_conversion_factor(base_currency: str, target_currency: str) -> float:
    """
    This function fetches the currency conversion factor between
    a give base currency and a target currency
    """
    url = f"URL/{base_currency}/{target_currency}
    response = requests.get(url)
    return resposne.json()

@tool
def convert(base_currency_value: int, conversion_rate: Annotated[float, InjectedToolArg]) -> float:
    """
    Given a currency conversion rate this function calculates the target
    currency from the give currency value
    """
    return base_currency_value * currency_rate

In [ ]:
# tool binding
llm = ChatOpenAI()
llm_with_tools = llm.bind_tools([get_conversion_factor, convert])

In [ ]:
messages = [HumanMessage('What is the conversion factor between USD and INR, convert 10 USD to inr')]
messages

In [ ]:
ai_message = llm_with_tools.invoke(messages)
ai.message.tool_calls

In [ ]:
import json
for tool_call in ai_message.tool_calls:
    print(tool_call)
    # exectute the first tool call to get the conv rate
    if tool_call['name'] == 'get_conversion_factor':
        tool_message1 = get_conversion_factor.invoke(tool_call)
        # fetch the conversion rate
        conversion_rate = json.loads(tool_message1.content)['conversion_rate']
        messages.append(tool_message1)
    # exectute the second tool using conversion rate from tool 1
    if tool_call['name'] == 'convert':
        # fetch the current argument
        tool_call['args']['conversion_rate'] = conversion_rate
        tool_message2 = convert.invoke(tool_call)
        messages.append(tool_message2)
